# Parse JDs, extract skills

But lowercase the data first

In [4]:
import os
import re
import json
from pprint import pprint
from bs4 import BeautifulSoup
from bs4.element import Comment

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import gensim
from pprint import pprint

from stop_words import get_stop_words

import codecs
import os
import time

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')

import spacy
import pandas as pd
import itertools as it
import en_core_web_sm

import spacy
nlp = spacy.load('en')



[nltk_data] Downloading package punkt to /home/goodgame/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/goodgame/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/goodgame/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/goodgame/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import codecs
import boto3
# from collections import defaultdict
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [3]:
s3 = boto3.resource('s3')

In [4]:
# Raghu's code to pull down jobs
BUCKET_NAME = 'tech-salary-project'

In [8]:
# %%time

# bucket = s3.Bucket(BUCKET_NAME)
# all_job_titles = []
# all_job_ids = []
# all_summaries = []
# for o in bucket.objects.all():
#     if o.key.startswith('salaries'):
#         continue
    
#     object = bucket.Object(o.key)
#     try:
#         lines = object.get()['Body'].read().decode('utf-8').splitlines()
#         for line in lines:
#             d = json.loads(line)
            
#             title = d['title']
#             jid = d['id']
#             summary = d['summary']
            
#             all_job_ids.append(jid)
#             all_job_titles.append(title)
#             all_summaries.append(summary)
#     except Exception as e:
#         continue

In [9]:
print(len(set(all_summaries)))

119326


In [10]:
print('Number of all job titles:', len(all_job_titles))
uniq_job_titles = set(all_job_titles)
print('Number of unique job titles:', len(uniq_job_titles))

print('Number of all job ids:', len(all_job_ids))
uniq_job_ids = set(all_job_ids)
print('Number of unique job ids:', len(uniq_job_ids))

uniq_job_ids = set(all_job_ids)
print('Number of unique job ids:', len(uniq_job_ids))

print('\nNumber of summaries:',len(all_summaries))
unique_summaries = set(all_summaries)
print('Number of unique summaries:', len(unique_summaries))

Number of all job titles: 269707
Number of unique job titles: 57106
Number of all job ids: 269707
Number of unique job ids: 106579
Number of unique job ids: 106579

Number of summaries: 269707
Number of unique summaries: 119326


## Use BS4 to convert the 'summary' field HTML into visible text, and load the files into memory

In [12]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def make_text(text):
    soup = BeautifulSoup(text, "html5lib")
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip().lower() for t in visible_texts) # Modified 27MAR18 to produce lowercase results

In [13]:
%%time
texts = []

for item in unique_summaries:
    texts.append(make_text(item))

CPU times: user 15min 20s, sys: 898 ms, total: 15min 21s
Wall time: 15min 21s


In [14]:
%%time
# write to disk as a single large file
with open('../additional_jobs/s3_visible_texts_02apr.txt', 'w') as outfile:
    for item in texts:
        outfile.write(item)
        outfile.write('\n')

CPU times: user 442 ms, sys: 420 ms, total: 862 ms
Wall time: 861 ms


In [1]:
texts = []

with open('../additional_jobs/s3_visible_texts_02apr.txt', 'r') as infile:
    for line in infile:
        line = line.strip()
        texts.append(line)

# Note: All texts are lowercase at this point.

In [143]:
# Optional preprocessing step... maybe see if this works later

# remove words that appear only once
# texts = [[word for word in document.lower().split() if word not in stop_words]
#          for document in parsed_dataset]

# from collections import defaultdict
# frequency = defaultdict(int)
# for text in texts:
#     for token in text:
#         frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [2]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    for review in filename:
        yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [7]:
%%time

with codecs.open('../additional_jobs/spacy_jds_concat_parsed.txt', 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(texts): # Changed 28MAR18 to include tokenization function call
        f.write(sentence + '\n')

In [8]:
unigram_sentences = LineSentence('../additional_jobs/spacy_jds_concat_parsed.txt')

In [9]:
# print some examples
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

this project manager will be involve in the entire product development lifecycle from concept to production work with cross functional team define project schedule budget and timeline to ensure successful outcome

duty and responsibility ensure successful product feature release by coordinate all phase of product development lead the cross functional team to support the product lifecycle facilitate the day to day activity need to successfully define and deliver the product pipeline responsible for create and manage detailed project plan drive product core team meeting report on project status issue and risk to stakeholder and facilitate the resolution of any project issue or delay work with cross functional team to define and implement tool and process for tracking dashboard and reporting assist with management of development partner and consultant as need assist with meet regulatory compliance and audit requirement provide ongoing feedback on process and delivery improvement other dut

In [10]:
%%time

bigram_model = Phrases(unigram_sentences)
bigram_model.save('../additional_jobs/spacy_bigram_model_all_PARSED')

CPU times: user 10.1 s, sys: 166 ms, total: 10.2 s
Wall time: 10.1 s


In [11]:
# load the finished model from disk
bigram_model = Phrases.load('../additional_jobs/spacy_bigram_model_all_PARSED')

In [12]:
%%time

with codecs.open('../additional_jobs/spacy_bigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 21.4 s, sys: 95.8 ms, total: 21.5 s
Wall time: 21.5 s


In [13]:
bigram_sentences = LineSentence('../additional_jobs/spacy_bigram_sentences_PARSED.txt')

In [16]:
# # print examples; certain bigrams are underlined
# for bigram_sentence in it.islice(bigram_sentences, 240, 250):
#     print(u' '.join(bigram_sentence))
#     print(u'')

In [17]:
%%time
trigram_model = Phrases(bigram_sentences)
trigram_model.save('../additional_jobs/spacy_trigram_model_all_PARSED')

CPU times: user 9.83 s, sys: 155 ms, total: 9.99 s
Wall time: 9.89 s


In [18]:
# load the finished model from disk
trigram_model = Phrases.load('../additional_jobs/spacy_trigram_model_all_PARSED')

In [19]:
%%time

with codecs.open('../additional_jobs/spacy_trigram_sentences_PARSED.txt', 'w', encoding='utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 19.7 s, sys: 136 ms, total: 19.8 s
Wall time: 19.8 s


In [20]:
trigram_sentences = LineSentence('../additional_jobs/spacy_trigram_sentences_PARSED.txt')

In [21]:
for trigram_sentence in it.islice(trigram_sentences, 240, 250):
    print(u' '.join(trigram_sentence))
    print(u'')

nexant support client through three distinct business_unit nexant energy software have develop and commercialize a number of industry lead software platform and application include nexant ienergy ® nexant grid360 ™ and nexant ihedge ® nexant utility service work with client to drive customer energy_saving ensure cost_effective program design implementation and management and promote customer engagement nexant energy chemical advisory provide strategic_planning project development competitor analysis market and price forecasting and nexantthink ™ a subscription program and online product portal

every_day nexant employee partner with customer to reimagine the world -PRON- live in and create a more sustainable_energy tomorrow

nexant inc

30_+_day_ago

company arvato digital service llc arvato fs usa requisition

-PRON- would 14981 overall job purpose report to the director analytic consult service and work_closely_with the global client lead project manager and sale manager the senior_d

In [22]:
# NLTK stopwords format is not a list; use this version
stopwords = get_stop_words('english')
additional_stopwords = ["\'s", 'or']
stopwords = stopwords + additional_stopwords
print(stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

#### Final preprocessing

In [26]:
%%time

with codecs.open('../additional_jobs/spacy_trigram_transformed_jds_all_PARSED.txt', 'w', encoding='utf_8') as f:
    for parsed_review in nlp.pipe(line_review(texts),
                                  batch_size=10000, n_threads=24):
        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in stopwords]

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


KeyboardInterrupt: 

In [ ]:
# probably another memory issue.

In [46]:
%%time

trigram_reviews = LineSentence('../additional_jobs/spacy_trigram_sentences_PARSED.txt')

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
trigram_dictionary.compactify()

trigram_dictionary.save('../additional_jobs/spacy_trigram_dict_all.dict')
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load('../additional_jobs/spacy_trigram_dict_all.dict')

CPU times: user 25 s, sys: 267 ms, total: 25.3 s
Wall time: 25.2 s


In [47]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [48]:
%%time

# generate bag-of-words representations for all JDs and save them as a matrix
MmCorpus.serialize('../additional_jobs/spacy_trigram_bow_corpus_all.mm',
                   trigram_bow_generator('../additional_jobs/spacy_trigram_sentences_PARSED.txt'))

CPU times: user 37 s, sys: 979 ms, total: 37.9 s
Wall time: 38 s
